# Build a nonoredundant library of classified Nematoda transposable elements
**Objective**: Build a fasta file of non redundant TEs from all of Nematoda  
**Input**: Denovo libs for the individual species built with RepeatModeler and classified with RepeatMasker  
**Output**: A single non-redundant Nematoda TEs library file formated for RepeatMasker   
**Strategy**:   

1. Concatenate all the individual libraries 
2. Make 80% identity clusters with usearch and collect centroid sequences
3. Classify unclassified centroid sequences with online Cencor

In [1]:
import TE, os
from Bio import SeqIO
from random import shuffle
from glob import glob

## Concatenate the *denovo* libs of all the speices

In [4]:
# Make a list of denove lib files
infiles = ['RepeatModeler/%s/consensi.fa.classified'%i 
           for i in TE.genome_codes_list('Genomes/', mode='++', code_file='genome_assembly_files_v3.csv')] 


redundant_lib_filname = 'Library/redundant_nematoda_lib'

# Concatenate the files
hndl = open(redundant_lib_filname,'wt')
for i in infiles:
    # To keep seq ids unique, we add the genome code
    code = i.split('/')[-2]
    records = list(SeqIO.parse(i,'fasta'))
    for r in records:
        r.id = "%s_%s"%(code,r.id)
    # write the file with the modified ids    
    SeqIO.write(records,'temp','fasta')
    # append it to the redundant lib
    hndl.write("%s\n"%open('temp','r').read())
    os.remove('temp')
hndl.close()



## Make a nono-redundant library
1. Sort the sequences by length
2. Cluster into 80% identity clusters.
3. Put the centroid sequence of each cluster in the non-redundant library file `Library/non_redundant_non_classified`. This file will have only the classifications made by RepeatMasker in the RepeatModeler pipeline.

In [5]:
# make sure to clean previous usearch runs
os.remove('seqs_sorted.fasta')
os.remove('results.uc')
os.remove('results.fasta')

TE.make_non_redundant_lib(redundant_lib_filname,
                          'Library/non_redundant_non_classified',
                          cluster_identity=0.8)

'Library/non_redundant_non_classified'

In [6]:
# How many cumulative families in all the lib files?
! grep -c ">" 'Library/redundant_nematoda_lib'

26163


In [7]:
# How many in the non-redundant file
! grep -c ">" 'Library/non_redundant_non_classified'

19873


A large proportion of multicopy TE families are found in a single nematode species (76%). They can still be present in other species, but not be detected by RepeatModeler because they are low copy number there.

## Split the library to < 2mb chunks for online [censor](http://www.girinst.org/censor/)

In [10]:
records = list(SeqIO.parse('Library/non_redundant_non_classified',
                           'fasta'))

shuffle(records) # so that the chunks are the same size
i = 0
while records:
    if len(records) > 1790:
        SeqIO.write(records[:1790],"Library/p%i"%i,'fasta')
        records = records[1790:]
        i += 1
    else:
        SeqIO.write(records,"Library/p%i"%i,'fasta')
        i += 1
        records = None

##[Censor](http://www.girinst.org/censor/) run
**Inputs**:files `p0`-`p11` in `Library`  
**Outputs**: files `Censor_results0`-`Censor_results11` in `Library`  
Default setting

## Parse the Censor output files

In [14]:
censor_classification = {}
for f in glob('Library/Censor_results*'):
    censor_classification.update(TE.parse_online_censor(f, 0.8, 300))

## Add superfamily classifiactions from Censor to unclassified centroid sequences

In [15]:
TE.put_censor_classification_in_repeatmodeler_lib('Library/non_redundant_non_classified',
                                                  censor_classification,
                                                  'Library/Nematoda.lib')

## Write the Censor results to a file

In [13]:
TE.print_online_censor(censor_classification, 'Library/censor_results_kept')